In [ ]:
# -*- coding: utf-8 -*-
from hyperwords import multi2mono as m2m
import os

In [ ]:
%run ./create_embeddings_lib.ipynb

In [ ]:
N=10  # Number of test samples to create

# Default file paths
training_corpus = 'bible'
path = './' + training_corpus + '/'
pairs_path = path + 'pairs'
enrich_corpus = 'wikipedia'
enrich_corpus_path = '../extract_scripts/' + enrich_corpus + '/data/ranked_150000/'
enrich_pairs_path = enrich_corpus_path + 'pairs'
combined_path = './' + training_corpus + '-' + enrich_corpus + '/'
combined_pairs_path = combined_path + 'pairs'
counts_path = combined_path + 'counts'
counts_contexts_path = combined_path + 'counts.contexts.vocab'
counts_words_path = combined_path + 'counts.words.vocab'
sgns_dir = 'sgns_enriched'
sgns_path = combined_path + sgns_dir + '/'

In [ ]:
# No need to refresh the bible pairs file every time
#[path, pairs_path] = create_pairs(path)

In [ ]:
# Create the pairs file for the enrich corpus
[enrich_path, enrich_pairs_path] = create_pairs(enrich_corpus_path)

In [ ]:
# Produce counts for the enriched pairs - used only for frequency distribution analysis, not for training
enrich_counts_path = create_counts(enrich_corpus_path, enrich_pairs_path)

In [ ]:
c2v.counts2vocab(enrich_counts_path)

In [ ]:
# Create an output directory for the combined pairs/counts
if not os.path.exists(combined_path):
    os.makedirs(combined_path)

In [ ]:
# Concatenate the pairs files together for the training corpus and the enrich corpus
filenames = [pairs_path, enrich_pairs_path]
with open(combined_pairs_path, 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [ ]:
# Produce counts from the combined pairs - then continue as per baseline process
counts_path = create_counts(combined_path, combined_pairs_path)

In [ ]:
c2v.counts2vocab(counts_path)

In [ ]:
for idx in range(0, N):
    print ('Starting sample: ' + str(idx))
    sgns_path = make_sgns_dir(combined_path, sgns_dir, idx)
    [sgns_contexts_path, sgns_words_path] = run_word2vecf(combined_path, sgns_path)
    # python2 hyperwords/multi2mono.py ./${1}/sgns/words ./${1}/sgns
    m2m.multi2mono(sgns_words_path, sgns_path)
    vecs_to_numpy(sgns_path)